In [ ]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from nemo.util import parse_colmap_point_cloud
from nemo.plotting import plot_3d_points
from nemo.dem import DEM

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

# Red Rocks

### COLMAP sparse point cloud

In [ ]:
path = '../data/redrocks/redrocks_points3D.txt'
points, colors = parse_colmap_point_cloud(path)

In [ ]:
# Plotting the points
fig = plot_3d_points(x=points[:,0], y=points[:,1], z=points[:,2])
fig.show()

In [ ]:
# Extract the points within range xyz in [-5, 5]
points = points[(points[:,0] > -5) & (points[:,0] < 5)]
points = points[(points[:,1] > -5) & (points[:,1] < 5)]
points = points[(points[:,2] > -5) & (points[:,2] < 5)]

fig = go.Figure()
fig = plot_3d_points(fig, x=points[:,0], y=points[:,1], z=points[:,2], markersize=1)
fig.show()

In [ ]:
# Estimate the ground plane
from nemo import geometry
plane = geometry.fit_plane(points)
plane

# We should use the nerfstudio dataparser

### DroneMapper DEM

In [ ]:
from osgeo import gdal

# Open the GeoTIFF file
# (checked by seems vertical datum metadata is missing)
dataset = gdal.Open('../data/redrocks/DEM32-DroneMapper.tif')

if dataset is None:
    print("Failed to open the GeoTIFF file")
    exit(1)

# Read raster data
band = dataset.GetRasterBand(1)  # Assuming it's a single-band raster
elevation_data = band.ReadAsArray()  

# Use geotransform parameters to calculate extent (use a local ENU coordinate frame with origin at center of DEM)
geotransform = dataset.GetGeoTransform()
top_left_x, pixel_width, x_rotation, top_left_y, y_rotation, pixel_height = geotransform
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# Calculate East and North extents
east_extent = cols * pixel_width
north_extent = rows * abs(pixel_height)

In [ ]:
dem = DEM(elevation_data, extent=(east_extent, north_extent))

In [ ]:
dem_ds = dem.downsample(10)
dem.handle_missing_data()
dem_ds.show()